In [1]:
import numpy as np
import pandas as pd
import pypsa  
import os 
import sys  
import datetime
from pyomo.environ import Constraint

file_name =  os.path.basename(sys.argv[0])
directory = os.path.dirname(os.path.realpath(file_name))+'/Output'
n_file = 'Output/' + [s for s in os.listdir(directory) if 'EU_Network_created' in s][0]

nz = pypsa.Network()
nz.import_from_netcdf('Output/nz_10hour.nc')
n1 = pypsa.Network()
n1.import_from_netcdf(n_file)

n_storage = pd.read_csv('Output/n_storage.csv',index_col=0)
n_storage.index = pd.to_datetime(n_storage.index)
Indices = np.load('Output/indices.npy')

INFO:pypsa.io:Imported network nz_10hour.nc has buses, carriers, generators, lines, links, loads, storage_units
INFO:pypsa.io:Imported network EU_Network_created_2020-11-02.nc has buses, carriers, generators, lines, links, loads, storage_units


In [2]:
#initialize dictionary of buses per country
Country2Bus = {}
for i in nz.buses['country'].unique():
    Country2Bus[i] = nz.buses[nz.buses['country'] == i].index

#add country tags to generators and storage units (for convenience)
nz.generators['country'] = np.array(nz.buses.loc[nz.generators['bus'],'country'])
nz.storage_units['country'] = np.array(nz.buses.loc[nz.storage_units['bus'],'country'])

#Get the list of storage units with significant standard deviation over the course of the year
Storage_units = nz.storage_units.loc[n_storage.std(axis=0)>100000].index

Lines = n1.lines.copy()

In [3]:
#Rules used for extra_functionality - gathered here because they are used at different times for different cases

def balance_rule(model,c):
    Country_Gens = nz.generators[nz.generators['country'] == c]
    Country_Gens = Country_Gens[~Country_Gens.index.str.contains('EmGenUp2|EmGenDown')].index
    Country_Storage = nz.storage_units[nz.storage_units['country'] == c].index 
    Gen_p = sum([model.generator_p[i,nz.snapshots[hour]] for i in Country_Gens])
    Storage_store = sum([model.storage_p_store[i,nz.snapshots[hour]] for i in Country_Storage])
    Storage_disp = sum([model.storage_p_dispatch[i,nz.snapshots[hour]] for i in Country_Storage])
    zonal_gen = nz.generators_t.p.loc[nz.snapshots[hour],Country_Gens].sum()
    zonal_storage = nz.storage_units_t.p.loc[nz.snapshots[hour],Country_Storage].sum()
    return  Gen_p + Storage_disp - Storage_store >= zonal_gen + zonal_storage

def storage_rule(model,su):
    final = n_storage.loc[nz.snapshots[hour],su]
    start = n_storage.loc[nz.snapshots[hour-1],su]
    if (final-start > 0) and ('hydro' in su):
        inflow = nz.storage_units_t.inflow.loc[nz.snapshots[hour],su]
        if start + inflow > final:
            return model.state_of_charge[su,nz.snapshots[hour]] >= final
        else:
            return model.state_of_charge[su,nz.snapshots[hour]] >= start + inflow
    else:
        return model.state_of_charge[su,nz.snapshots[hour]] >= final
    
def extra_functionality(network,snapshots):
    model = network.model
    model.balance = Constraint(nz.buses['country'].unique(),rule=balance_rule)
    model.storage = Constraint(Storage_units,rule=storage_rule)

In [4]:
n1.storage_units['cyclic_state_of_charge'] = False

#enable emergency balancing gens
n1.generators.loc[n1.generators.index.str.contains('EmGenUp2'),'p_max_pu'] = 1
n1.generators.loc[n1.generators.index.str.contains('EmGenDown'),'p_min_pu'] = -1

iterations = 2
Dispatch = {}
Storage = {}

for k in np.arange(iterations):
    
    #initialize generator output dataframe
    Dispatch[k] = pd.DataFrame(index=n1.snapshots[Indices],columns=n1.generators.index) 
    Storage[k] = pd.DataFrame(index=n1.snapshots[Indices],columns=n1.storage_units.index) 
    
    for i in n1.buses['country'].unique():

        Country_Lines = Lines.loc[[(Lines.loc[j,'bus0'] in Country2Bus[i])|(Lines.loc[j,'bus1'] in Country2Bus[i]) for j in Lines.index]].index
        Country_Storage = n1.storage_units.loc[nz.storage_units['country'] == i].index
        Country_Gens = n1.generators[nz.generators['country'] == i].index        
        
        n1.lines.s_nom = np.Inf
        n1.lines.s_nom[Country_Lines] = Lines.loc[Country_Lines,'s_nom']        
        n1.storage_units_t.state_of_charge_set.drop(columns = n1.storage_units_t.state_of_charge_set.columns,inplace=True)
        n1.generators_t.p_set.drop(columns = n1.generators_t.p_set.columns,inplace=True) 
        
        if k == 0:
            n1.storage_units_t.state_of_charge_set = pd.concat([n1.storage_units_t.state_of_charge_set,nz.storage_units_t.state_of_charge],axis=1)
            n1.generators_t.p_set = pd.concat([n1.generators_t.p_set,nz.generators_t.p],axis=1)
        else:
            n1.storage_units_t.state_of_charge_set = pd.concat([n1.storage_units_t.state_of_charge_set,Storage[k-1]],axis=1)
            n1.generators_t.p_set = pd.concat([n1.generators_t.p_set,Dispatch[k-1]],axis=1)
        
        n1.storage_units_t.state_of_charge_set.drop(columns = Country_Storage,inplace=True)
        n1.generators_t.p_set.drop(columns = Country_Gens,inplace=True)
        n1.generators_t.p_set.drop(columns = n1.generators_t.p_set.columns[n1.generators_t.p_set.columns.str.contains('EmGenUp2|EmGenDown')],inplace=True)
            
        for j in Indices:
            print(k, ':',i,':',j)
            print(datetime.datetime.now())
            hour = j
            timesteps = np.array([hour])
            n1.storage_units['state_of_charge_initial'] = n_storage.iloc[hour-1]
            n1.lopf(n1.snapshots[timesteps],solver_name='gurobi',extra_functionality=extra_functionality)
        
        Dispatch[k].loc[:,Country_Gens] = n1.generators_t.p.loc[n1.snapshots[Indices],Country_Gens]
        Storage[k].loc[:,Country_Storage] = n1.storage_units_t.state_of_charge.loc[n1.snapshots[Indices],Country_Storage]

C:\Users\brytni.johnston\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



0 : PT : 2723
2020-11-09 18:28:45.854693


INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `kirchhoff` formulation
INFO:pypsa.opf:Solving model using gurobi
INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x22368_copy
  Lower bound: 2611061.396711419
  Upper bound: 2611061.396711419
  Number of objectives: 1
  Number of constraints: 6508
  Number of variables: 22368
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 22368
  Number of nonzeros: 70851
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination mes

INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `kirchhoff` formulation
INFO:pypsa.opf:Solving model using gurobi
INFO:pypsa.opf:Optimization successful


...

INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `kirchhoff` formulation
INFO:pypsa.opf:Solving model using gurobi
INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x22368_copy
  Lower bound: 1117950.7824008456
  Upper bound: 1117950.7824008456
  Number of objectives: 1
  Number of constraints: 6508
  Number of variables: 22368
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 22368
  Number of nonzeros: 70851
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination m

INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `kirchhoff` formulation
INFO:pypsa.opf:Solving model using gurobi
INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x22368_copy
  Lower bound: 2091266.9109814726
  Upper bound: 2091266.9109814726
  Number of objectives: 1
  Number of constraints: 6508
  Number of variables: 22368
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 22368
  Number of nonzeros: 70851
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination m

KeyboardInterrupt: 

In [5]:
n1.lines = Lines
n1.storage_units_t.state_of_charge_set.drop(columns = n1.storage_units_t.state_of_charge_set.columns,inplace=True)
n1.generators_t.p_set.drop(columns = n1.generators_t.p_set.columns,inplace=True) 
n1.generators_t.p_set = pd.concat([n1.generators_t.p_set,Dispatch[0]],axis=1)
n1.storage_units_t.state_of_charge_set = pd.concat([n1.storage_units_t.state_of_charge_set,Storage[0]],axis=1)
n1.generators_t.p_set.drop(columns = n1.generators_t.p_set.columns[n1.generators_t.p_set.columns.str.contains('EmGen')],inplace=True)

# Run optimization for desired timesteps
for i in Indices: 

    print('Hour: ',i)
    print(datetime.datetime.now())
    hour = i
    timesteps = np.array([hour])
    
    n1.storage_units['state_of_charge_initial'] = n_storage.iloc[hour-1]
    n1.lopf(nz.snapshots[timesteps],solver_name='gurobi')

print(datetime.datetime.now())
n1.export_to_netcdf('Output/n1it0_10hour.nc')

n1.lines_t.p0.iloc[Indices].to_csv('Output/n1it0_lines_10h.csv')
n1.generators_t.p.iloc[Indices].to_csv('Output/n1it0_gen_10h.csv')
n1.storage_units_t.p.iloc[Indices].to_csv('Output/n1it0_storage_10h.csv')

Hour:  2723
2020-11-05 02:11:44.774207


INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `kirchhoff` formulation
INFO:pypsa.opf:Solving model using gurobi
INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x22368_copy
  Lower bound: 790413512.4216822
  Upper bound: 790413512.4216822
  Number of objectives: 1
  Number of constraints: 14865
  Number of variables: 22368
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 22368
  Number of nonzeros: 61185
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination me

INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `kirchhoff` formulation
INFO:pypsa.opf:Solving model using gurobi
INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x22368_copy
  Lower bound: 267943301.76127413
  Upper bound: 267943301.76127413
  Number of objectives: 1
  Number of constraints: 14865
  Number of variables: 22368
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 22368
  Number of nonzeros: 61185
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination 

...

INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `kirchhoff` formulation
INFO:pypsa.opf:Solving model using gurobi
INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x22368_copy
  Lower bound: 6372487586.1234665
  Upper bound: 6372487586.1234665
  Number of objectives: 1
  Number of constraints: 14865
  Number of variables: 22368
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 22368
  Number of nonzeros: 61185
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination 

INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `kirchhoff` formulation
INFO:pypsa.opf:Solving model using gurobi
INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x22368_copy
  Lower bound: 1106173.7492113304
  Upper bound: 1106173.7492113304
  Number of objectives: 1
  Number of constraints: 14865
  Number of variables: 22368
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 22368
  Number of nonzeros: 61185
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination 

INFO:pypsa.io:Exported network n1it0_10hour.nc has generators, storage_units, loads, carriers, buses, links, lines


In [6]:
n1.lines = Lines
n1.storage_units_t.state_of_charge_set.drop(columns = n1.storage_units_t.state_of_charge_set.columns,inplace=True)
n1.generators_t.p_set.drop(columns = n1.generators_t.p_set.columns,inplace=True) 
n1.generators_t.p_set = pd.concat([n1.generators_t.p_set,Dispatch[1]],axis=1)
n1.storage_units_t.state_of_charge_set = pd.concat([n1.storage_units_t.state_of_charge_set,Storage[1]],axis=1)
n1.generators_t.p_set.drop(columns = n1.generators_t.p_set.columns[n1.generators_t.p_set.columns.str.contains('EmGen')],inplace=True)
# Run optimization for desired timesteps
for i in Indices: 

    print('Hour: ',i)
    print(datetime.datetime.now())
    hour = i
    timesteps = np.array([hour])
    
    n1.storage_units['state_of_charge_initial'] = n_storage.iloc[hour-1]
    n1.lopf(nz.snapshots[timesteps],solver_name='gurobi')

print(datetime.datetime.now())
n1.export_to_netcdf('Output/n1it1_10hour.nc')

n1.lines_t.p0.iloc[Indices].to_csv('Output/n1it1_lines_10h.csv')
n1.generators_t.p.iloc[Indices].to_csv('Output/n1it1_gen_10h.csv')
n1.storage_units_t.p.iloc[Indices].to_csv('Output/n1it1_storage_10h.csv')

Hour:  2723
2020-11-05 02:17:03.062582


INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `kirchhoff` formulation
INFO:pypsa.opf:Solving model using gurobi
INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x22368_copy
  Lower bound: 790413659.7391824
  Upper bound: 790413659.7391824
  Number of objectives: 1
  Number of constraints: 14865
  Number of variables: 22368
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 22368
  Number of nonzeros: 61185
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination me

INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `kirchhoff` formulation
INFO:pypsa.opf:Solving model using gurobi
INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x22368_copy
  Lower bound: 267938376.8070243
  Upper bound: 267938376.8070243
  Number of objectives: 1
  Number of constraints: 14865
  Number of variables: 22368
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 22368
  Number of nonzeros: 61185
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination me

...

INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `kirchhoff` formulation
INFO:pypsa.opf:Solving model using gurobi
INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x22368_copy
  Lower bound: 6374863565.140119
  Upper bound: 6374863565.140119
  Number of objectives: 1
  Number of constraints: 14865
  Number of variables: 22368
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 22368
  Number of nonzeros: 61185
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination me

INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `kirchhoff` formulation
INFO:pypsa.opf:Solving model using gurobi
INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x22368_copy
  Lower bound: 1115529.0958197764
  Upper bound: 1115529.0958197764
  Number of objectives: 1
  Number of constraints: 14865
  Number of variables: 22368
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 22368
  Number of nonzeros: 61185
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination 

INFO:pypsa.io:Exported network n1it1_10hour.nc has generators, storage_units, loads, carriers, buses, links, lines
